In [3]:
import json
import collections
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import pandas as pd
import nltk
from sklearn import preprocessing
import time
from datetime import datetime
from datetime import timedelta

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
# with open("data/baseline_data.csv",'w') as f:
#     csv_write = csv.writer(f)
#     top_20_words = ["china", "billion", "products", "democrats", "great","dollars","tariffs","country", "mueller","border",\
#                      "president", "congressman", "people", "korea", "party", "years", "farmers", "going", "trade", "never"]

#     csv_head = ["created_time","favorite_count", "retweet_count"] + top_20_words 
#     csv_write.writerow(csv_head)
    
#     month_map = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8,\
#             'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
#     for tweet in data:
#         timel =  tweet['created_at'].split()
#         nomral_time = timel[5] + '.' + str(month_map[timel[1]])+'.'+ timel[2]+' '+timel[3]
#         row = [nomral_time, tweet['favorite_count'], tweet['retweet_count']]
#         for w in top_20_words:
#             row.append(1) if w in tweet['processed_text'] else row.append(0)
#         csv_write.writerow(row)

In [26]:
df1= pd.read_csv('data/baseline_data.csv')
senti_data = pd.read_csv('data/sentimentAnalysis/senti.csv')
senti_data['time'] = senti_data['time'].apply(lambda x: x[:-6])
df2 = senti_data[['time', 'positive_count', 'negative_count']]
df = pd.merge(df1, df2, left_on = 'created_time', right_on='time').drop(['time'], axis=1)
df.dtypes

created_time       object
favorite_count      int64
retweet_count       int64
china               int64
billion             int64
products            int64
democrats           int64
great               int64
dollars             int64
tariffs             int64
country             int64
mueller             int64
border              int64
president           int64
congressman         int64
people              int64
korea               int64
party               int64
years               int64
farmers             int64
going               int64
trade               int64
never               int64
score               int64
tb_polarity       float64
tb_subjetivity    float64
positive_count      int64
negative_count      int64
dtype: object

In [29]:
df['begin_time'] = df['created_time'].apply(lambda x: x[:-2]+'00')
df['created_time'] = df['created_time'].apply(lambda x: pd.to_datetime(x))
df['begin_time'] = df['begin_time'].apply(lambda x: pd.to_datetime(x))
df['end_time'] = df['begin_time'] + timedelta(minutes=5)

In [30]:
df.head(2)

,created_time,favorite_count,retweet_count,china,billion,products,democrats,great,dollars,tariffs,...,going,trade,never,score,tb_polarity,tb_subjetivity,positive_count,negative_count,begin_time,end_time
0,2019-11-16 15:18:27,15011,4301,0,0,0,0,0,0,0,...,0,0,0,-1,0.383333,0.377778,3,4,2019-11-16 15:18:00,2019-11-16 15:23:00
1,2019-11-16 13:56:23,0,9149,0,0,0,0,0,0,0,...,0,0,0,-1,0.000000,0.000000,0,1,2019-11-16 13:56:00,2019-11-16 14:01:00


In [31]:
data_econ = pd.read_csv('data/all_cleaned_SPX_data.csv')
data_econ['datetime'] = pd.to_datetime(data_econ['datetime'])
df_econ = data_econ[['datetime', 'close']]

In [32]:
df_merge1 = pd.merge(df, df_econ, left_on = 'begin_time', right_on='datetime').drop(['datetime'], axis=1)
df_merge2 = pd.merge(df_merge1, df_econ, left_on = 'end_time', right_on='datetime').drop(['datetime'], axis=1)
df_merge2.head()

,created_time,favorite_count,retweet_count,china,billion,products,democrats,great,dollars,tariffs,...,never,score,tb_polarity,tb_subjetivity,positive_count,negative_count,begin_time,end_time,close_x,close_y
0,2019-11-07 15:43:29,114187,20211,0,0,0,0,0,0,0,...,0,1,0.178788,0.351515,1,0,2019-11-07 15:43:00,2019-11-07 15:48:00,3080.94,3082.54
1,2019-11-07 15:41:53,72619,18328,0,0,0,0,0,0,0,...,0,-2,0.161174,0.349053,1,3,2019-11-07 15:41:00,2019-11-07 15:46:00,3082.59,3081.32
2,2019-11-07 15:27:57,83333,24007,0,0,0,0,0,0,0,...,0,-3,-0.142857,0.750000,1,4,2019-11-07 15:27:00,2019-11-07 15:32:00,3082.41,3083.63
3,2019-11-07 15:18:53,65540,14962,0,0,0,0,0,0,0,...,0,1,0.000000,0.000000,1,0,2019-11-07 15:18:00,2019-11-07 15:23:00,3082.46,3082.57
4,2019-11-07 15:16:15,132078,35482,0,0,0,0,0,0,0,...,1,-2,-0.208333,0.458333,0,2,2019-11-07 15:16:00,2019-11-07 15:21:00,3084.20,3080.28


In [33]:
df_merge2['label'] = (df_merge2['close_y'] > df_merge2['close_x'])*1
df_all = df_merge2.drop(['created_time','begin_time','end_time', 'close_x','close_y'], axis=1)

In [34]:
df_all = df_all.drop(['tb_polarity','tb_subjetivity'], axis=1)
df_all.head()

,favorite_count,retweet_count,china,billion,products,democrats,great,dollars,tariffs,country,...,party,years,farmers,going,trade,never,score,positive_count,negative_count,label
0,114187,20211,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
1,72619,18328,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2,1,3,0
2,83333,24007,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-3,1,4,1
3,65540,14962,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
4,132078,35482,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,-2,0,2,0


In [35]:
X_train, X_test, y_train, y_test =train_test_split(df_all.loc[:, df_all.columns != 'label'], 
                                                         df_all.label, test_size=0.2, 
                                                         random_state = 109)

In [36]:
min_max_scaler = MinMaxScaler().fit(X_train)
xtrain_scaled = min_max_scaler.transform(X_train.values)
xtest_scaled = min_max_scaler.transform(X_test.values)
X_train_scaled = pd.DataFrame(xtrain_scaled, columns = X_train.columns, index = X_train.index)
X_test_scaled = pd.DataFrame(xtest_scaled, columns = X_test.columns, index = X_test.index)

In [37]:
Logistic_model = LogisticRegression(C=100000)
Logistic_model.fit(X_train_scaled, y_train)
y_train_pred_logistic = Logistic_model.predict(X_train_scaled)
y_test_pred_logistic = Logistic_model.predict(X_test_scaled)

/Users/yuan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [38]:
print("Logistic train accuracy: {}\nLogistic test accuracy: {}"\
      .format(accuracy_score(y_train_pred_logistic, y_train),accuracy_score(y_test_pred_logistic, y_test)))

Logistic train accuracy: 0.5234956670328329
Logistic test accuracy: 0.49829184968277207


In [39]:
CV_logistic = LogisticRegressionCV()
CV_logistic.fit(X_train_scaled, y_train)

/Users/yuan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [40]:
print("CV logistic regression\nTraining accuracy: {}\nTesting accuracy: {}"\
     .format(accuracy_score(CV_logistic.predict(X_train_scaled), y_train), accuracy_score(CV_logistic.predict(X_test_scaled), y_test)))

CV logistic regression
Training accuracy: 0.5275234956670328
Testing accuracy: 0.5017081503172279


In [41]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=20, n_estimators=1000)
rf.fit(X_train_scaled,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
print("Random Forest\nTraining accuracy: {}\nTesting accuracy: {}"\
     .format(accuracy_score(rf.predict(X_train_scaled), y_train), accuracy_score(rf.predict(X_test_scaled), y_test)))

Random Forest
Training accuracy: 0.9166361528133773
Testing accuracy: 0.5324548560273304


In [52]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_train, rf.predict(X_train_scaled))
metrics.auc(fpr, tpr)

0.9159378828418842